# Project 3: Binary Classification of subreddits
# Notebook 1: Introduction and Extraction

## Problem Statement

Mental health issues have been on a high in recent times, with increasing cases of as a result of factors like post traumatic stress disorder (PTSD), bullying (real-life or cyber) and suicides. All these lead to depression which is a cause of concern for the government. 

Being part of a research team in the Institute of Mental Health (IMH), my team is tasked to come up a solution that can accurately predict and classify a post on reddit according to the subreddits (depression and forever alone). Both topics are commonly thought to be similar but they are not. Forever alone is a phrase or term used by a single person to express his loneliness at not having a significant other; or in broader perspective, lack of friends. It is also used more humorously at times as a slang or internet meme. This term is linked to people claiming to be "depressed" but in fact they are not.

On the other hand, depression is a serious matter and more often than not, one do not even realize that they actually have depression. It is also extremely challenging to determine if a person has depression by relying on behaviorial cues, even for a highly qualified psychiatrist. Furthermore, if the depressed person is introverted, he/she might not want to share his problems openly to his/her family or friends or even seek professional help in fear being stigmatized by society. As such, the person might turn to reddit, an online forum, to express his/her personal feelings and thoughts with other users in the same subreddit annoymously. He/she might feel more comfortable in that way. That being said, a user with depression might get confused over the terms depression and forever alone in thinking they are the same , thus posting in the forever alone subreddit rather than the depression subreddit. 

Hence, we aim to come up with a model that is able to accurately classify a post to the respective subreddits (depression/forever alone). Our model also aims to check for misclassification of post by diving deeper into the post, unraveling the false positive and identifying top 10 words of each subreddit. Hence, we are able to provide early detection of potential depression cases along with identify existing cases. We can then look to get them the help they need. The target audience of this project is everyone, especially those working in the psychological and healthcare departments.

## Executive Summary

The objective of this project is to execute different binary classification algorithms/models and find the best model over several attempts. Firstly, i will extract about 2000 posts for the depression and forever alone subreddits. I will then clean the data to remove duplicates, missing values and outliers. 

Next up is the exploratory data analysis (EDA) where i analyze the data, identifying relationships  and displaying their distrbution.  This would give a quick view of what are the deciding variables in relation to classifying the respective subreddits.

It is then followed by pre-processing, where i began to prepare my data for the modeling process. This step includes lemmatizing my words to the base root, removing default english stopwords and my own customised stopwords which consists of top common words that exist in both datasets.

As for modeling, my initial models will be without any adjustments to hyperparameters. My second attempt will be to run the models with tuned hyperparamaters and find the most accurate model. Lastly, i will use my final model and evaluate it further to conclude.

I will be using count and tfidf vectorizers to tokenise the variable columns. I decided to make use of 4 different classifiers (Logistic Regression, Random Forest, K Nearest Neighbours and Multinomial Naive Bayes) to create different variations. I will then evaluate their respective scores along with the confusion matrix to evaluate and select my final model.

The final model selected was the Multinimial Naive Bayes model equipped with the tfidf vectorizer.

## Imports

In [1]:
# Imports
import requests
import pandas as pd
import numpy as np
import random
import time
from time import sleep

### Extraction

First, i will go on to pull posts from the 2 subreddits using PushShift API and store them as dataframes. I will create a function to execute the extraction. Pushshift API limits me to 100 posts per pull. I have included a sleep timer to avoid getting blocked by reddit for repeated pulling in short intervals. 

The function also include the number of times the pull is executed and the cumulative number of posts pulled.

In [2]:
# This code was adapted and referenced from my groupmates Mark and Chin Xia who guided me on the process

def pull_reddit(subreddit):
    ''' Function to pull posts from subreddits.
        Output returns the post pulled. '''
    df = pd.DataFrame()
    url = 'https://api.pushshift.io/reddit/search/submission'
    bef_counter = '0d'
    i = 0
    while 2000 - len(df) > 0:
        time.sleep(np.random.randint(1,5)) # Setting a sleep timer
        params = {
            'subreddit': subreddit,
            'size': 100,
            'before': bef_counter,
            'unique': 'selftext'}
        res = requests.get(url, params)
        print(f'Pull Number {i}, Status Code: {res.status_code}')
        if res.status_code == 200: # Status_code check
            data = res.json()
            new_df = pd.DataFrame(data['data'])
            cleaned_df = new_df.drop(new_df[
                (new_df['selftext'] == '[removed]') | # Dropping removed posts
                (new_df['selftext'] == '[deleted]') | # Dropping deleted posts
                (new_df['selftext'] == '') | # Dropping empty posts
                (new_df['is_video'] == True) # Dropping non-text posts
            ].index)
            bef_counter = str(i*10)+'d'
            df = pd.concat([df, cleaned_df], axis = 0)
            i += 1
            print(f'Number of post pulled : {len(df)}')
        else:
            print(f'Loop {i} failed, pausing script')
            time.sleep(np.random.randint(2,3))
    return df

In [3]:
depression = pull_reddit('depression')
depression

Pull Number 0, Status Code: 200
Number of post pulled : 87
Pull Number 1, Status Code: 200
Number of post pulled : 174
Pull Number 2, Status Code: 200
Number of post pulled : 256
Pull Number 3, Status Code: 200
Number of post pulled : 348
Pull Number 4, Status Code: 200
Number of post pulled : 438
Pull Number 5, Status Code: 200
Number of post pulled : 519
Pull Number 6, Status Code: 200
Number of post pulled : 604
Pull Number 7, Status Code: 200
Number of post pulled : 690
Pull Number 8, Status Code: 200
Number of post pulled : 768
Pull Number 9, Status Code: 200
Number of post pulled : 850
Pull Number 10, Status Code: 200
Number of post pulled : 936
Pull Number 11, Status Code: 200
Number of post pulled : 1016
Pull Number 12, Status Code: 200
Number of post pulled : 1104
Pull Number 13, Status Code: 200
Number of post pulled : 1186
Pull Number 14, Status Code: 200
Number of post pulled : 1267
Pull Number 15, Status Code: 200
Number of post pulled : 1353
Pull Number 16, Status Code: 2

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,whitelist_status,wls,removed_by_category,post_hint,preview,edited,author_flair_background_color,author_flair_text_color,banned_by,author_cakeday
0,[],False,DirectBar2805,None,[],None,text,t2_96ueyj4o,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,SkippAndTripp,None,[],None,text,t2_9hug5dy6,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,mo-91,None,[],None,text,t2_zcssk,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,[],False,Immatt55,None,[],None,text,t2_mimzj,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[],False,cherry91804,None,[],None,text,t2_3gnvl233,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,[],False,iNpAiNAsHtOn,None,[],None,text,t2_8z3m7365,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,[],False,CuzIamcreep,None,[],None,text,t2_8niuc2wl,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,[],False,imsorry333,None,[],None,text,t2_8fvwsbbp,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,[],False,mybearisjelly,None,[],None,text,t2_5vnys074,False,False,...,no_ads,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
forever_alone = pull_reddit('ForeverAlone')
forever_alone

Pull Number 0, Status Code: 200
Number of post pulled : 66
Pull Number 1, Status Code: 200
Number of post pulled : 132
Pull Number 2, Status Code: 200
Number of post pulled : 192
Pull Number 3, Status Code: 200
Number of post pulled : 255
Pull Number 4, Status Code: 200
Number of post pulled : 317
Pull Number 5, Status Code: 200
Number of post pulled : 375
Pull Number 6, Status Code: 200
Number of post pulled : 446
Pull Number 7, Status Code: 200
Number of post pulled : 497
Pull Number 8, Status Code: 200
Number of post pulled : 558
Pull Number 9, Status Code: 200
Number of post pulled : 625
Pull Number 10, Status Code: 200
Number of post pulled : 669
Pull Number 11, Status Code: 200
Number of post pulled : 728
Pull Number 12, Status Code: 200
Number of post pulled : 774
Pull Number 13, Status Code: 200
Number of post pulled : 825
Pull Number 14, Status Code: 200
Number of post pulled : 878
Pull Number 15, Status Code: 200
Number of post pulled : 928
Pull Number 16, Status Code: 200
Nu

In [ ]:
print(depression.shape)
print(forever_alone.shape)

In [ ]:
depression = depression[['subreddit', 'selftext','title', 'author']]
depression = pd.DataFrame(depression)
depression.head()

In [ ]:
forever_alone = forever_alone[['subreddit', 'selftext','title', 'author']]
forever_alone = pd.DataFrame(forever_alone)
forever_alone.head()

In [ ]:
#depression.to_csv('./data/depression.csv', index=False)
#forever_alone.to_csv('./data/forever_alone.csv', index=False)